# Devesh's Implementation of a Convolutional Neural Network to classify the MNIST Fashion Dataset

## Setup

In [1]:
# Import statements
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

2023-07-05 18:29:20.276016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.12.0-rc0


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255, x_test/255

# Channels Dimms
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# Batching and shuffling the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Building the Model

In [3]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, (3,3), activation="relu")
        self.maxPooling1 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(64, activation="relu")
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxPooling1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [4]:
# Choosing and optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# This is the metrics for loss and accuracy
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [5]:
# Training the Model

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training = True only needed for dropout and other like layers
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)    
    train_accuracy(labels, predictions)

In [6]:
# Test the model
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [10]:
EPOCHS = 15
for epoch in range(EPOCHS):
    # Reset metrics at the start of each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels, in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )


Epoch 1, Loss: 0.04829656332731247, Accuracy: 98.22999572753906, Test Loss: 0.3939874768257141, Test Accuracy: 91.15999603271484
Epoch 2, Loss: 0.042035289108753204, Accuracy: 98.50166320800781, Test Loss: 0.4211900234222412, Test Accuracy: 91.43000030517578
Epoch 3, Loss: 0.03642602264881134, Accuracy: 98.69166564941406, Test Loss: 0.4842073619365692, Test Accuracy: 90.94999694824219
Epoch 4, Loss: 0.03357263281941414, Accuracy: 98.83333587646484, Test Loss: 0.4705022871494293, Test Accuracy: 91.86000061035156
Epoch 5, Loss: 0.030054103583097458, Accuracy: 98.92666625976562, Test Loss: 0.5027772188186646, Test Accuracy: 90.86000061035156
Epoch 6, Loss: 0.03192662447690964, Accuracy: 98.88333129882812, Test Loss: 0.5155314207077026, Test Accuracy: 90.66000366210938
Epoch 7, Loss: 0.025920387357473373, Accuracy: 99.11500549316406, Test Loss: 0.5607569217681885, Test Accuracy: 91.18000030517578
Epoch 8, Loss: 0.022757308557629585, Accuracy: 99.25833129882812, Test Loss: 0.536743342876434

In [8]:
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  640       
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  692288    
                                                                 
 dense_1 (Dense)             multiple                  650       
                                                                 
Total params: 693,578
Trainable params: 693,578
Non-trainable params: 0
____________________________________________________